In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os

In [2]:
data=pd.read_csv('./Stress-Lysis.csv')
data.head()  # displays the first five rows of the dataframe

,Humidity,Temperature,Step_count,Stress_Level
0,21.33,90.33,123,1
1,21.41,90.41,93,1
2,27.12,96.12,196,2
3,27.64,96.64,177,2
4,10.87,79.87,87,0


In [3]:
data.shape

(2001, 4)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
import os

class StopTrainingOnHighAccuracy(Callback):
    def __init__(self, monitor='val_accuracy', target=1.0, save_path="final_model.h5"):
        super().__init__()
        self.monitor = monitor
        self.target = target
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        
        if current is not None and current >= self.target:
            print(f"\nEpoch {epoch + 1}: {self.monitor} reached {self.target}. Stopping training.")
            # Save the model
            self.model.save(self.save_path)
            self.model.stop_training = True
# Separate features and target
X = data.drop(columns=['Stress_Level']).values  # Features
y = data['Stress_Level'].values  # Target
y = to_categorical(y, num_classes=3)
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model with smaller kernel size
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_save_path = "final_model.h5"
stop_training_callback = StopTrainingOnHighAccuracy(
    monitor='val_accuracy', 
    target=1.0, 
    save_path=model_save_path
)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[stop_training_callback])


# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

Epoch 1/50
50/50 [==============================] - 0s 4ms/step - loss: 3.4712 - accuracy: 0.5169 - val_loss: 0.5216 - val_accuracy: 0.7232
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4665 - accuracy: 0.7594 - val_loss: 0.4271 - val_accuracy: 0.7880
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3828 - accuracy: 0.8181 - val_loss: 0.3456 - val_accuracy: 0.8653
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3453 - accuracy: 0.8650 - val_loss: 0.3115 - val_accuracy: 0.9127
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3065 - accuracy: 0.8850 - val_loss: 0.2675 - val_accuracy: 0.9451
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2969 - accuracy: 0.8769 - val_loss: 0.3007 - val_accuracy: 0.8579
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2501 - accuracy: 0.9187 - val_loss: 0.2198 - val_accuracy: 0.9327
Epoch 8/50
50/50 [==

C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def celsius_to_fahrenheit(celsius):
    fahrenheit = (celsius * 9/5) + 32
    return fahrenheit

new_data = np.array([[21.33, celsius_to_fahrenheit(32.4), 123]])  # Ganti dengan nilai fitur yang diinginkan

# Pastikan jumlah fitur sesuai dengan yang digunakan saat training
# Preprocessing sama seperti saat training
# new_data_scaled = scaler.transform(new_data)  # Scaling menggunakan scaler yang sama
# new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], new_data_scaled.shape[1], 1))

# Melakukan prediksi
prediction = model.predict(new_data)
predicted_class = np.argmax(prediction, axis=1)  # Mendapatkan kelas prediksi

# Convert predicted class back to original label
# predicted_label = label_encoder.inverse_transform(predicted_class)

print("Predicted Stress Level:", predicted_class)

1/1 [==============================] - 0s 17ms/step
Predicted Stress Level: [2]


In [6]:
from tensorflow.keras.models import load_model
best_model = load_model(model_save_path)